In [2]:
import time
import copy
import pandas as pd
import torch
from torch.autograd import Variable
from densenet import densenet169
from utils import plot_training, n_p, get_count
from train import train_model, get_metrics
from pipeline import get_study_level_data, get_dataloaders

In [3]:
# #### load study level dict data
study_data = get_study_level_data(study_type='XR_WRIST')


100%|███████████████████████████████████████████████████████████████████████████████| 207/207 [00:01<00:00, 128.18it/s]


In [4]:
# #### Create dataloaders pipeline
data_cat = ['train', 'valid'] # data categories
dataloaders = get_dataloaders(study_data, batch_size=1)
dataset_sizes = {x: len(study_data[x]) for x in data_cat}

In [5]:
# #### Build model
# tai = total abnormal images, tni = total normal images
tai = {x: get_count(study_data[x], 'positive') for x in data_cat}
tni = {x: get_count(study_data[x], 'negative') for x in data_cat}
Wt1 = {x: n_p(tni[x] / (tni[x] + tai[x])) for x in data_cat}
Wt0 = {x: n_p(tai[x] / (tni[x] + tai[x])) for x in data_cat}

In [6]:
print('tai:', tai)
print('tni:', tni, '\n')
print('Wt0 train:', Wt0['train'])
print('Wt0 valid:', Wt0['valid'])
print('Wt1 train:', Wt1['train'])
print('Wt1 valid:', Wt1['valid'])

tai: {'train': 3987, 'valid': 295}
tni: {'train': 5769, 'valid': 364} 

Wt0 train: tensor([0.4087])
Wt0 valid: tensor([0.4476])
Wt1 train: tensor([0.5913])
Wt1 valid: tensor([0.5524])


In [7]:
class Loss(torch.nn.modules.Module):
    def __init__(self, Wt1, Wt0):
        super(Loss, self).__init__()
        self.Wt1 = Wt1
        self.Wt0 = Wt0
        
    def forward(self, inputs, targets, phase):
        loss = - (self.Wt1[phase] * targets * inputs.log() + self.Wt0[phase] * (1 - targets) * (1 - inputs).log())
        return loss

In [8]:
model = densenet169(pretrained=True)
#model = model.cuda()

#model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet169', pretrained=True)
#model.eval()


C:\Users\paula\Documents\Repositorio\Test\densenet.py:115: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [9]:

criterion = Loss(Wt1, Wt0)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=1, verbose=True)

C:\Users\paula\miniforge3\envs\tfg\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [10]:
dataset_sizes

{'train': 3460, 'valid': 237}

In [13]:
dataloaders.items()

dict_items([('train', <torch.utils.data.dataloader.DataLoader object at 0x000002D8B94424B0>), ('valid', <torch.utils.data.dataloader.DataLoader object at 0x000002D8A92096A0>)])

In [9]:
# #### Train model
model = train_model(model, criterion, optimizer, dataloaders, scheduler, dataset_sizes, num_epochs=5)

Train batches: 3460
Valid batches: 237 

Epoch 1/5
----------


IndexError: tuple index out of range

In [ ]:
torch.save(model.state_dict(), 'models/model.pth')

In [ ]:
get_metrics(model, criterion, dataloaders, dataset_sizes)